In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('data/data Procena darovitosti.xlsx', sheet_name='Sheet1')

In [3]:
df.head()

,RB,naspol,nasss,nastrs,nasvl,naslm,nasvp,nastk,nasmr,nasinter,...,ucetk,ucemr,uceinter,uceintra,ucepriro,ucefil,rpm,vs,shv,Rnasvl
0,1,1,3,2,44,33,55,48,57,55,...,42,55,49,54,40,39,30,13,17,10.0
1,2,1,3,2,50,40,44,42,38,60,...,37,36,65,63,44,33,36,12,16,30.0
2,3,1,3,2,42,42,36,43,40,56,...,36,34,45,37,29,25,36,13,16,6.0
3,4,1,3,1,50,60,32,28,34,46,...,55,60,45,45,51,37,31,12,10,30.0
4,5,1,3,1,46,45,43,61,53,51,...,54,41,52,57,51,46,33,5,13,15.0


In [4]:
df.set_index('RB', inplace=True)

## Strategije detekcije anomalija

U nastavku biće navedene i ukratko objašnjene strategije i algoritmi koje planiram da koristim za detekciju anomalija nad učitanim skupom podataka.

### 1. Univarijantne strategije

Ova grupa strategija jeste znatno jednostavnija od naredne. Pri ovom pristupu posmatramo svaki atribut posebno, ne i njihove kombinacije. Možemo probati 2 jako slična pristupa:
1. Identifikacija anomalija na osnovu z-score
2. Identifikacija anomalija na osnovu IQR-a (interkvartalni opseg)

Oba pristupa su dobra samo u slučaju da su varijable normalno raspoređene, a rade loše se `iskošenim` raspodelama. S obzirom da u našem skupu postoji par desno-iskošenih raspodela, ove metode ćemo probati samo kao početnu tačku: postavićemo veliki prag kako bi identifikovali samo one koji imaju znatno veće odstupanje po nekom atributu (to obično budu neke greške u upisivanju podatka).

Evo intuicije iza oba pristupa:

#### 1.1. Z-score based anomaly detection
Prvi pristup se zasniva na tome da, pod pretpostavkom da je varijabla normalno raspoređena, posmatramo anomalije kao tačke sa najvećim z-score-om (z score = koliko standardnih devijacija je tačka udaljena od srednje vrednosti). 

#### 1.2. IQR based anomaly detection
Drugi pristup posmatra interkvartalni opseg (iqr = razlika između prvog i trećeg kvartila). Sve tačke koje su ispod `quartile_1 - iqr` ili iznad `quartile_3 + iqr`, posmatraju se kao anomalije.

Više o ovim pristupima može se pročitati u [ovom blogu](http://colingorrie.github.io/outlier-detection.html).

### 2. Multivarijantne strategije

Grubo, ove algoritme možemo podeliti na tri osnovne grupe:
1. Algoritmi zasnovani na najbližim susedima
2. Algoritmi zasnovani na klasterovanju
3. Algoritmi zasnovani na ansamblima

#### 2.1. KNN-based anomaly detector

Proračunava udaljenost tačke od svojih k-najbližih suseda. Za svaku tačku se kao njen anomaly_score koristi udaljenost od k suseda. Može se koristiti: 
- Udaljenost od najdaljeg od k suseda
- Srednja udaljenost
- Medijana udaljenosti

Više o algoritmu i Python implementaciji može se pročitati u citiranim radovima i [dokumentaciji PyOD paketa](https://pyod.readthedocs.io/en/latest/pyod.models.html#module-pyod.models.knn).

#### 2.2. Angle-Based Outlier Detection (ABOD)

[Više o algoritmu](https://imada.sdu.dk/~zimek/publications/KDD2008/KDD08-ABOD.pdf) Može se reći da pripada grupi algoritama zasnovanim na susedima

Intuicija: Posmatramo veze svake observacije sa ostalima tako što, umesto da računamo samo udaljenost, projektujemo vektore. Ako je spektar vektora veliki obzervacija nije outlier (u svim pravcima u odnosu na njega ima drugih zapisa). Ako su svi vektori ka jednoj strani onda je outlier.

Postoji implementacija u Pythonu. Više u [dokumentaciji.](https://pyod.readthedocs.io/en/latest/pyod.models.html#module-pyod.models.abod)

#### 2.3. Clustering-based anomaly detector

Osnovna ideja: Podeliti podatke u klastere i kao anomaly_score koristiti udaljenost od centroida. 

Postoji Python implementacija i o njoj se više može pročitati [ovde](https://pyod.readthedocs.io/en/latest/pyod.models.html#module-pyod.models.cblof). 

Međutim poznato je da je k-means veoma pristrasan kada radi sa korelisanim podacima. Ideja je da pored klasične implementacije u Pythonu, probamo sledeći prilagođeni algoritam:
1. Uraditi PCA
2. Probati više clustering modela za različiti broj klastera nad glavnim komponentama
3. Izabrati broj klastera
4. Pustiti konačan klastering model i kao anomaly_score koristiti udaljenost od centroida
5. Nakon uočenih anomalija, vratiti se u početni prostor

O kombinaciji PCA i klasterovanja može se pročitati u [ovom radu](https://ieeexplore.ieee.org/document/7012911).

#### 2.4. Isolation Forest

Ansambli u službi otkrivanja anomalija. Ovo je prilagođena verzija algoritma Random Forest. Pravi slučajnu šumu stabala odlučivanja i kao anomaly_score posmatra koliko je obzervacija izolovana od ostalih (tako što meri sa još koliko drugih tačaka se našla u listu).

[Više o algoritmu](http://www.extended-cognition.com/2018/11/15/multivariate-outlier-detection-with-isolation-forests/) [Originalni Rad](https://cs.nju.edu.cn/zhouzh/zhouzh.files/publication/tkdd11.pdf)

[Python implementacija](https://pyod.readthedocs.io/en/latest/pyod.models.html#module-pyod.models.iforest)